# Calculate station profile additional mixing effect

In [ ]:
import numpy as np
import netCDF4 as nc
import pandas as pd
from datetime import date
from datetime import datetime
import xarray as xr
import glob
import modules.mapping_functions as mf

#### Parameters:

In [ ]:
# domain dimensions:
imin, imax = 1479, 2179
jmin, jmax = 159, 799

# results to plot:
folder_ref_2015      = f'/data/brogalla/run_storage/Mn-reference-202110/ANHA12_ref-2015_20211012/'
folder_ref_2009      = f'/data/brogalla/run_storage/Mn-reference-202110/ANHA12_ref-2009_20211012/'

# colours:
c_continent    = '#ce9169'
c_glacier      = '#36ab92'
c_other        = '#a6a6a6'
land_color     = "#8b7765"
profile_colors = ['#e54141', '#33c7ff']

#### Load files:

In [ ]:
def calculate_average(folder_ref_2009, folder_ref_2015):
    # Calculate average over data files during cruise period:
    # 2009 GEOTRACES: 29 August - 8 September
    files_2009      = np.sort(glob.glob(f'{folder_ref_2009}ANHA12_EXH006_5d_20090101_20091231*'))
    start_date_2009 = datetime.strptime('2009-08-29', '%Y-%m-%d')
    end_date_2009   = datetime.strptime('2009-09-08', '%Y-%m-%d')

    file_list_2009 = []
    for file in files_2009:
        if (datetime.strptime(file.split('/')[-1][42:50],'%Y%m%d') >= start_date_2009) & \
                (datetime.strptime(file.split('/')[-1][51:59],'%Y%m%d') < end_date_2009):
            file_list_2009.append(file)
    dmn_2009 = np.zeros((50,700,640))
    days = 0
    for file in file_list_2009:
        ds = xr.open_dataset(f"{file}")
        dmn_2009 = dmn_2009 + ds['dissolmn'].values[0,:,:,:]

    mn_dis_ref_2009 = dmn_2009 / len(file_list_2009)

    # 2015 GEOTRACES: 9 August - 15 September
    files_2015      = np.sort(glob.glob(f'{folder_ref_2015}ANHA12_EXH006_5d_20150101_20151231*'))
    start_date_2015 = datetime.strptime(f'2015-08-09', '%Y-%m-%d')
    end_date_2015   = datetime.strptime(f'2015-09-15', '%Y-%m-%d')

    file_list_2015 = []
    for file in files_2015:
        if (datetime.strptime(file.split('/')[-1][42:50],'%Y%m%d') >= start_date_2015) & \
                    (datetime.strptime(file.split('/')[-1][51:59],'%Y%m%d') < end_date_2015):
            file_list_2015.append(file)

    dmn_2015 = np.zeros((50,700,640))
    days = 0
    for file in file_list_2015:
        ds = xr.open_dataset(f"{file}")
        dmn_2015 = dmn_2015 + ds['dissolmn'].values[0,:,:,:]

    mn_dis_ref_2015 = dmn_2015 / len(file_list_2015)
    
    return mn_dis_ref_2009, mn_dis_ref_2015

In [ ]:
mn_ref_2009, mn_ref_2015           = calculate_average(folder_ref_2009, folder_ref_2015)

Model results

In [ ]:
# Load 2015 Mn reference run results
model_ref_2015 = nc.Dataset(f'{folder_ref_2015}ANHA12_EXH006_2015_monthly.nc')
lon_model    = np.array(model_ref_2015.variables['nav_lon'])
lat_model    = np.array(model_ref_2015.variables['nav_lat'])
depths_model = np.array(model_ref_2015.variables['deptht'])

Meshes

In [ ]:
# ANHA12 grid
mesh       = nc.Dataset('/ocean/brogalla/GEOTRACES/data/ANHA12/ANHA12_mesh1.nc')
mesh_bathy = np.array(mesh.variables['hdept'][0])
tmask      = np.array(mesh.variables['tmask'])[0,:,imin:imax,jmin:jmax]
Z_masked   = np.ma.masked_where((tmask > 0.1), tmask) 
e3t        = np.array(mesh.variables['e3t_0'])

Observations

In [ ]:
# Concentration of dMn in the Beaufort Sea in 2009 (data from Nari Sim's thesis: http://hdl.handle.net/2429/67889
IPY = pd.read_csv('/ocean/brogalla/GEOTRACES/data/Nari_Sim_dMn_data.csv')
IPY_names_full = IPY['Station name'].astype('string').values
IPY_depths     = IPY['Depth [m]'].astype('float').values
IPY_dMn_nmolkg = IPY['DMn [ nmol/kg]'].astype('float').values
IPY_lons_full  = -1*IPY['Longitude'].astype('float').values
IPY_lats_full  = IPY['Latitude'].astype('float').values

# Unique entries (maintaining order)
IPY_lons  = np.array(list(dict.fromkeys(IPY_lons_full)))
IPY_lats  = np.array(list(dict.fromkeys(IPY_lats_full)))
IPY_names = np.array(list(dict.fromkeys(IPY_names_full)))

# Convert nmol / kg to nmol / L
IPY_density             = IPY['Density'].astype('float').values - 1000 # kg / L
IPY_depths_cruise_sheet = IPY['Depth [m] cruise sheet'].astype('float').values
IPY_dMn_nmolL           = IPY_dMn_nmolkg * (IPY_density+1000)/1000

In [ ]:
# Concentration of dMn in 2015 (data from Colombo et al., 2020)
Manuel = pd.read_csv('/ocean/brogalla/GEOTRACES/data/DMn_nmolL-1_BR-edited.csv')
Manuel_station_names = Manuel['Station'].astype('string')
Manuel_depths        = Manuel['Depth'].astype('float')
Manuel_dMn_nmolkg    = Manuel['nmol kg-1'].astype('float').values
Manuel_dMn_nmolL     = Manuel['nmol L-1'].astype('float').values
Manuel_salinity      = Manuel['Salinity'].astype('float').values

GEOTRACES = pd.read_csv('/ocean/brogalla/GEOTRACES/data/Pb-paper-data.csv')
stn_names = GEOTRACES['Station names'].dropna().astype('str')
lons      = GEOTRACES['station lon'].astype('float').dropna().values
lats      = GEOTRACES['Station lat'].astype('float').dropna().values

##### Define functions

In [ ]:
def station_info_2015(name):
    # Gather and sort data from 2015

    dMn_obs   = Manuel_dMn_nmolL[np.array(Manuel_station_names==name).astype('bool')]
    depth_obs = Manuel_depths[np.array(Manuel_station_names==name).astype('bool')].values
    obs_e3t   = np.diff(depth_obs, prepend=0) # size of vertical boxes

    lon_station = lons[stn_names==name]
    lat_station = lats[stn_names==name]

    model_stni, model_stnj = mf.find_indeces_vector(lon_station, lat_station, lon_model, lat_model)
    model_stni = int(model_stni); model_stnj = int(model_stnj);
    
    e3t_stn = np.ma.masked_where((tmask[:,model_stni,model_stnj] < 0.1), e3t[0,:,model_stni+imin,model_stnj+jmin])

    mn_ref = np.ma.masked_where((tmask[:,model_stni,model_stnj] < 0.1), mn_ref_2015[:, model_stni, model_stnj]*10**9)
    
    return obs_e3t, dMn_obs, e3t_stn, mn_ref

def station_info_2009(name):
    # Gather and sort data from 2009

    dMn_obs   = IPY_dMn_nmolL[np.array(IPY_names_full==name).astype('bool')]
    depth_obs = IPY_depths_cruise_sheet[np.array(IPY_names_full==name).astype('bool')]
    obs_e3t   = np.diff(depth_obs, prepend=0) # size of vertical boxes

    lon_station = IPY_lons[IPY_names==name]
    lat_station = IPY_lats[IPY_names==name]

    model_stni, model_stnj = mf.find_indeces_vector(lon_station, lat_station, lon_model, lat_model)
    model_stni = int(model_stni); model_stnj = int(model_stnj);
    
    e3t_stn = np.ma.masked_where((tmask[:,model_stni,model_stnj] < 0.1), e3t[0,:,model_stni+imin,model_stnj+jmin])

    mn_ref = np.ma.masked_where((tmask[:,model_stni,model_stnj] < 0.1), mn_ref_2009[:, model_stni, model_stnj]*10**9)
    
    return obs_e3t, dMn_obs, e3t_stn, mn_ref

In [ ]:
def depth_integrate(name, year=''):
    # Integrate profiles over full depth:
    
    if year=='2015':
        obs_e3t_stn, obs_dMn_stn, model_e3t_stn, model_dMn_stn = station_info_2015(name)
    elif year=='2009':
        obs_e3t_stn, obs_dMn_stn, model_e3t_stn, model_dMn_stn = station_info_2009(name)
    else:
        print('Not a year with Mn evaluation data')
        return np.empty([]), np.empty([])
    
    # depth-integrated Mn content:
    obs_totaldMn_stn   = obs_e3t_stn*obs_dMn_stn
    model_totaldMn_stn = model_e3t_stn*model_dMn_stn
    
    # average Mn in water column:
    obs_ave_dMn   = sum(obs_totaldMn_stn)/sum(obs_e3t_stn)
    model_ave_dMn = np.ma.sum(model_totaldMn_stn)/np.ma.sum(model_e3t_stn)
    
    return obs_ave_dMn, model_ave_dMn

def depth_integrate_subsurface(name, year=''):
    # Integrate profiles over subsurface depth:
    
    if year=='2015':
        obs_e3t_stn, obs_dMn_stn, model_e3t_stn, model_dMn_stn = station_info_2015(name)
    elif year=='2009':
        obs_e3t_stn, obs_dMn_stn, model_e3t_stn, model_dMn_stn = station_info_2009(name)
    else:
        print('Not a year with Mn evaluation data')
    
    subsurface_depth = 52
    obs_depth   = np.cumsum(obs_e3t_stn) 
    model_depth = np.cumsum(model_e3t_stn)
    
    # depth-integrated Mn content:
    obs_totaldMn_stn   = sum(obs_e3t_stn[obs_depth < subsurface_depth]*obs_dMn_stn[obs_depth < subsurface_depth])
    model_totaldMn_stn = np.ma.sum(model_e3t_stn[model_depth < subsurface_depth]*model_dMn_stn[model_depth < subsurface_depth])
    
    # average Mn in water column:
    obs_ave_dMn   = obs_totaldMn_stn/np.amax(obs_depth[obs_depth < subsurface_depth])
    model_ave_dMn = model_totaldMn_stn/np.amax(model_depth[model_depth < subsurface_depth])
    
    return obs_ave_dMn, model_ave_dMn

def total_dMn_subsurface(name, year=''):
    # Sum content in subsurface: 
    
    if year=='2015':
        obs_e3t_stn, obs_dMn_stn, model_e3t_stn, model_dMn_stn = station_info_2015(name)
    elif year=='2009':
        obs_e3t_stn, obs_dMn_stn, model_e3t_stn, model_dMn_stn = station_info_2009(name)
    else:
        print('Not a year with Mn evaluation data')
    
    subsurface_depth = 52
    obs_depth   = np.cumsum(obs_e3t_stn) 
    model_depth = np.cumsum(model_e3t_stn)
    
    # depth-integrated Mn content:
    obs_totaldMn_stn   = sum(obs_e3t_stn[obs_depth < subsurface_depth]*obs_dMn_stn[obs_depth < subsurface_depth])
    model_totaldMn_stn = np.ma.sum(model_e3t_stn[model_depth < subsurface_depth]*model_dMn_stn[model_depth < subsurface_depth])
    
    print(f'Total dMn at {name}: obs (upper {np.amax(obs_depth[obs_depth < subsurface_depth]):.1f}m): '+
          f'{obs_totaldMn_stn:.2f}, model (upper {np.amax(model_depth[model_depth < subsurface_depth]):.1f}m): {model_totaldMn_stn:.2f}')
    
    return obs_totaldMn_stn, model_totaldMn_stn

#### Calculate:

In [ ]:
obs_e3t_CAA9, obs_dMn_CAA9, model_e3t_CAA9, model_dMn_CAA9 = station_info_2015('CAA9')

In [ ]:
obs_ave_dMn_CAA4, model_ave_dMn_CAA4 = depth_integrate_subsurface('CAA4', year='2015')
obs_ave_dMn_CAA5, model_ave_dMn_CAA5 = depth_integrate_subsurface('CAA5', year='2015')
obs_ave_dMn_CAA6, model_ave_dMn_CAA6 = depth_integrate_subsurface('CAA6', year='2015')
obs_ave_dMn_CAA9, model_ave_dMn_CAA9 = depth_integrate_subsurface('CAA9', year='2015')

In [ ]:
obs_total_dMn_CAA4, model_total_dMn_CAA4 = total_dMn_subsurface('CAA4', year='2015')
obs_total_dMn_CAA5, model_total_dMn_CAA5 = total_dMn_subsurface('CAA5', year='2015')
obs_total_dMn_CAA6, model_total_dMn_CAA6 = total_dMn_subsurface('CAA6', year='2015')
obs_total_dMn_CAA9, model_total_dMn_CAA9 = total_dMn_subsurface('CAA9', year='2015')

In [ ]:
stn_list = ['CAA1', 'CAA2', 'CAA3', 'CAA4', 'CAA5', 'CAA6', 'CAA7', 'CAA8', 'CAA9']
combined_difference = 0; combined_observed=0; combined_modelled=0;
print('Subsurface averaged Mn concentration:')
for stn in stn_list:
    obs_ave_dMn, model_ave_dMn = depth_integrate_subsurface(stn, year='2015')
    print(f'{stn}: observed {obs_ave_dMn:.2f}, modelled {model_ave_dMn:.2f}, difference {model_ave_dMn-obs_ave_dMn:.2f}' )
    combined_difference += model_ave_dMn - obs_ave_dMn
    combined_observed += obs_ave_dMn
    combined_modelled += model_ave_dMn

print(f'Average observed subsurface Mn concentration: {combined_observed/len(stn_list):.2f}')
print(f'Average modelled subsurface Mn concentration: {combined_modelled/len(stn_list):.2f}')
print(f'Average difference between modelled and observed subsurface Mn concentration: {combined_difference/len(stn_list):.2f}')

In [ ]:
stn_list = ['CB1', 'CB2', 'CB3', 'CB4']
combined_difference = 0; combined_observed=0; combined_modelled=0;
print('Subsurface averaged Mn concentration:')
for stn in stn_list:
    obs_ave_dMn, model_ave_dMn = depth_integrate_subsurface(stn, year='2015')
    print(f'{stn}: observed {obs_ave_dMn:.2f}, modelled {model_ave_dMn:.2f}, difference {model_ave_dMn-obs_ave_dMn:.2f}' )
    combined_difference += model_ave_dMn - obs_ave_dMn
    combined_observed += obs_ave_dMn
    combined_modelled += model_ave_dMn

print(f'Average observed subsurface Mn concentration: {combined_observed/len(stn_list):.2f}')
print(f'Average modelled subsurface Mn concentration: {combined_modelled/len(stn_list):.2f}')
print(f'Average difference between modelled and observed subsurface Mn concentration: {combined_difference/len(stn_list):.2f}')

In [ ]:
stn_list = ['L1','L1.1','L1.5','L2','L3','S4']
combined_difference = 0; combined_observed=0; combined_modelled=0;
print('Subsurface averaged Mn concentration:')
for stn in stn_list:
    obs_ave_dMn, model_ave_dMn = depth_integrate_subsurface(stn, year='2009')
    print(f'{stn}: observed {obs_ave_dMn:.2f}, modelled {model_ave_dMn:.2f}, difference {model_ave_dMn-obs_ave_dMn:.2f}' )
    combined_difference += model_ave_dMn - obs_ave_dMn
    combined_observed += obs_ave_dMn
    combined_modelled += model_ave_dMn

print(f'Average observed subsurface Mn concentration: {combined_observed/len(stn_list):.2f}')
print(f'Average modelled subsurface Mn concentration: {combined_modelled/len(stn_list):.2f}')
print(f'Average difference between modelled and observed subsurface Mn concentration: {combined_difference/len(stn_list):.2f}')

In [ ]:
stn_list = ['CAA1', 'CAA2', 'CAA3', 'CAA4', 'CAA5', 'CAA6', 'CAA7', 'CAA8', 'CAA9']
combined_difference = 0
print('Full water column averaged Mn concentration:')
for stn in stn_list:
    obs_ave_dMn, model_ave_dMn = depth_integrate(stn, year='2015')
    print(f'{stn}: observed {obs_ave_dMn:.2f}, modelled {model_ave_dMn:.2f}, difference {model_ave_dMn-obs_ave_dMn:.2f}' )
    combined_difference += model_ave_dMn - obs_ave_dMn
    
print(f'Average difference between modelled and observed full water column Mn concentration: {combined_difference/len(stn_list):.2f}')